In [1]:
from transformers import Trainer,get_linear_schedule_with_warmup,RobertaTokenizer,BertForSequenceClassification,BitsAndBytesConfig
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
import psutil
from sklearn.model_selection import train_test_split
from peft import prepare_model_for_kbit_training
from peft import get_peft_model
import time
from collections import defaultdict
import math

In [2]:
def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters: {total:,}")
    print(f"Percentage of trainable params: {100 * trainable / total:.2f}%")

## Custom RoBERTa class needs work

In [3]:
from transformers import RobertaForSequenceClassification
import torch.nn as nn

class RobertaForSequenceClassificationWithCustomHead(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.custom_layer = nn.Sequential(
            nn.Linear(config.num_labels, 64),
            nn.ReLU(),
            nn.Linear(64, config.num_labels)
        )

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        accepted_keys = {
            "input_ids", "attention_mask", "labels", "token_type_ids",
            "position_ids", "head_mask", "inputs_embeds", "output_attentions",
            "output_hidden_states", "return_dict"
        }
        filtered_kwargs = {k: v for k, v in kwargs.items() if k in accepted_keys}

        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            **filtered_kwargs
        )

        logits = outputs.logits

        # Move custom layer to correct device if needed
        if next(self.custom_layer.parameters()).device != logits.device:
            self.custom_layer.to(logits.device)

        custom_logits = self.custom_layer(logits)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(custom_logits, labels)

        return type(outputs)(
            loss=loss,
            logits=custom_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
from transformers import RobertaTokenizerFast
from transformers import RobertaForSequenceClassification

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=True
)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Number of available GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available.")
device
model = RobertaForSequenceClassificationWithCustomHead.from_pretrained("roberta-base", num_labels=15, quantization_config=bnb_config, device_map="auto")
print(model.config)
print_trainable_parameters(model)

# Custom tokenizer trained on OOL tokens
custom_tokenizer = RobertaTokenizer.from_pretrained('./tokenizer') # Try to remove Fast?

Number of available GPUs: 1
GPU Name: NVIDIA GeForce RTX 3080


AttributeError: 'Parameter' object has no attribute 'SCB'

In [ ]:
custom_vocab = set(custom_tokenizer.get_vocab().keys())

new_tokens = list(custom_vocab)

print(f"Found {len(new_tokens)} new tokens to use.")

Found 30522 new tokens to use.


In [ ]:
model.resize_token_embeddings(len(custom_tokenizer))

Embedding(30522, 768, padding_idx=1)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    task_type=TaskType.SEQ_CLS,     # or SEQ_2_SEQ_LM, TOKEN_CLS, etc.
    r=8,
    lora_alpha=32,
    lora_dropout=0.2,
    target_modules=["query", "key", "value"],  # Adjust for RoBERTa
    use_dora=True
)

model = get_peft_model(model, config)
print_trainable_parameters(model)
#model.to(device)

Trainable parameters: 1,072,143
Total parameters: 68,101,357
Percentage of trainable params: 1.57%


In [ ]:
from datasets import load_dataset
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

data = pd.read_pickle('./saved_data/encoded_data.pck')
le = LabelEncoder()
data['target'] = le.fit_transform(data['Attack_type'])

data2 = pd.read_csv(".\data\Edge-IIoTset dataset\Selected dataset for ML and DL\ML-EdgeIIoT-dataset.csv")
columns = list(data2.columns[:-2])

In [ ]:
with open("column_name_encoded_data.txt","w") as f:
    for _, row in data.iterrows():
        tokens = row['encoded_PPFLE'].split(" ")
        formatted = []
        for idx, (col, tok) in enumerate(zip(columns, tokens)):
            end_char = "." if idx == len(tokens) - 1 else ";"
            formatted.append(f"{col}: {tok}{end_char}")
        row_string = " ".join(formatted)
        f.write(row_string + "\n")
#'<s>','<pad>','</s>','<unk>','<mask>'

In [ ]:
with open("column_name_encoded_data.txt", "r") as f:
    prompt_lines = f.readlines()

# Strip newline characters
prompt_lines = [prompt_line.strip() for prompt_line in prompt_lines]
prompt_data = {
    "encoded_PPFLE_prompt": prompt_lines, 'Attack_type': data['Attack_type'], 'Attack_label': data['Attack_label'], 'target': data['target']
}
prompt_df = pd.DataFrame(prompt_data)
prompt_df.sample(15)

,encoded_PPFLE_prompt,Attack_type,Attack_label,target
132736,frame.time: 52bbee57f7584edca9f7d865ddef44d977...,DDoS_UDP,1,4
12466,frame.time: 4997fcf6a96efbd5474a93c4d32118150f...,Ransomware,1,10
39192,frame.time: 26ac88ea1b357cde53d46d57122f996ee6...,DDoS_HTTP,1,1
3672,frame.time: 27f16c64e331b309d5ec22bcd9e100cf70...,Ransomware,1,10
107530,frame.time: 4196d9b45389faf499d21056daa33a761f...,Normal,0,7
154241,frame.time: 7183de64eda221ea7d3ad7d13898372895...,DDoS_ICMP,1,2
44393,frame.time: 17193dc56ca53e85bdd173130cfab194fa...,DDoS_TCP,1,3
91755,frame.time: 764e556221846b2c53d3f648af5afa5ebf...,Backdoor,1,0
53665,frame.time: 0adcf1af4c1e93e58919a108f1d6193c56...,DDoS_TCP,1,3
84186,frame.time: dced56621bda7629676a8f06e7cd2433f5...,Vulnerability_scanner,1,13


In [ ]:
train_set = prompt_df.sample(frac=0.7,random_state=42).reset_index(drop=True)

remaining = prompt_df.drop(train_set.index).reset_index(drop=True)

test_set = remaining.sample(frac=0.5,random_state=42).reset_index(drop=True)

val_set = remaining.drop(test_set.index).reset_index(drop=True)

print(train_set.shape,val_set.shape,test_set.shape)

(110460, 4) (23670, 4) (23670, 4)


In [ ]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_set, test_set = train_test_split(prompt_df, test_size=test_ratio,stratify=prompt_df.iloc[:,-1], random_state=42)
train_set, val_set = train_test_split(train_set, test_size=val_ratio/(val_ratio+train_ratio),stratify=train_set.iloc[:,-1], random_state=42)

In [ ]:
train_set.shape,val_set.shape,test_set.shape

((110460, 4), (23670, 4), (23670, 4))

In [ ]:
TARGET_LIST = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP',
                'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning',
                'Ransomware', 'SQL_injection', 'Uploading', 'Vulnerability_scanner',
                'XSS']

In [ ]:
prompt_df[prompt_df['Attack_type']=='Uploading'].head(5)

,encoded_PPFLE_prompt,Attack_type,Attack_label,target
13140,frame.time: 05293357ba325a590027981a9b59eb748a...,Uploading,1,12
13141,frame.time: da0992898db14d190c3b6f7c2c6c658259...,Uploading,1,12
13142,frame.time: 4d5b89f0a6797f41a5d0ee5b7b1cfe1327...,Uploading,1,12
13143,frame.time: 732fcefab7fb269fd8ae7bd4d3b3e28ec3...,Uploading,1,12
13144,frame.time: 9a7d459b7d8a0e5c070cb85c3b1bd0824a...,Uploading,1,12


class CustomDataset(Dataset):
  def __init__(self,encodings,df,max_len):
    self.encodings = encodings
    self.df = df
    self.max_len=max_len
    self.targets = self.df['target'].tolist()

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    target = self.targets[idx]
    encoding = self.encodings[idx]

    return {
        'input_ids':encoding['input_ids'].flatten(),
        'attention_mask':encoding['attention_mask'].flatten(),
        'targets':torch.tensor(target,dtype=torch.long)
    }

from datasets import load_dataset, Dataset
from transformers import RobertaTokenizerFast
import torch

# Define tokenization function
def tokenize_function(row):
    return custom_tokenizer(
        row["encoded_PPFLE_prompt"],               # replace with your actual column
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

# Tokenize entire dataset
train_set_enc = (Dataset.from_pandas(train_set)).map(tokenize_function, batched=True)
val_set_enc = (Dataset.from_pandas(val_set)).map(tokenize_function, batched=True)
test_set_enc = (Dataset.from_pandas(test_set)).map(tokenize_function, batched=True)

# Save to disk (recommended)
train_set_enc.save_to_disk('lora_roberta_train_encodings.pt')
val_set_enc.save_to_disk('lora_roberta_val_encodings.pt')
test_set_enc.save_to_disk('lora_roberta_test_encodings.pt')

In [ ]:
from datasets import load_dataset, Dataset
from datasets import load_from_disk

train_set_enc = load_from_disk('lora_roberta_train_encodings.pt')
print(len(train_set_enc))
val_set_enc = load_from_disk('lora_roberta_val_encodings.pt')
print(len(val_set_enc))
test_set_enc = load_from_disk('lora_roberta_test_encodings.pt')
print(len(test_set_enc))

110460
23670
23670


In [ ]:
def set_dataset_format(dataset):
    dataset = dataset.rename_column('target', "labels")
    dataset.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"]  # include "labels" column
    )
    return dataset

train_dataset = set_dataset_format(train_set_enc)
val_dataset = set_dataset_format(val_set_enc)
test_dataset = set_dataset_format(test_set_enc)

In [ ]:
MAX_LEN=512
BATCH_SIZE=32

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=0

)

val_loader = DataLoader(
    val_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=0

)

test_loader = DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=0

)

test_data = next(iter(train_loader))

print(test_data['input_ids'].shape)

labels = train_dataset["labels"]
print(f"Labels min, max: {labels.min(), labels.max()}")
print(f"Labels dtype: {labels.dtype}")

for batch in train_loader:
    print(f"batch.keys = {batch.keys()}, ")
    break

print(train_dataset['input_ids'][0])

torch.Size([32, 512])
Labels min, max: (tensor(0), tensor(14))
Labels dtype: torch.int64
batch.keys = dict_keys(['labels', 'input_ids', 'attention_mask']), 
tensor([   0,   74,   86,   69,   81,   73,   18,   88,   77,   81,   73,   30,
         423,  322,   74, 2149, 1648,   74,   24,   72,  327,   71,  517,   72,
        3352, 1806,   71, 2128,  279,  347,  407,  410,   31,  225,   77,   84,
          18,   87,   86,   71,   67,   76,   83,   87,   88,   30,  383,  274,
          28,   72,   29,  306,  883,  307,   27,   69,  753,   71,  874,   69,
         308,  268,   20,  317,   28,  270,   20,  263,  884,   31,  225,   77,
          84,   18,   72,   87,   88,   67,   76,   83,   87,   88,   30,  809,
         263,   20,   71,   29,  797,  344,  268,   25,  795,   26,   74,  807,
          72,   23,   73,   31,  739,   86,   84,   18,   72,   87,   88,   18,
          84,   86,   83,   88,   83,   67,   77,   84,   90,   24,   30,  620,
         618,  325,  621,  278,  321,   72,

In [ ]:
def load_chkpt(model,version):
  return model.load_state_dict(torch.load(f"./saved_model/securityRoBERTa{version}.0.pt"))

In [ ]:
from transformers import TrainerCallback
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.epoch_train_losses = []
        self.epoch_train_accuracies = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        
        if "loss" in logs:
            self.epoch_train_losses.append(logs["loss"])

        if "eval_accuracy" in logs:
            self.epoch_train_accuracies.append(logs["eval_accuracy"])

In [ ]:
from transformers import Trainer
import torch
import numpy as np

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.history = defaultdict(list)
        self.best_val_acc = 0.0

    def evaluate_and_save(self, epoch, model_version):
        # Evaluate on validation set
        metrics = self.evaluate()
        print(metrics.keys())
        val_acc = metrics["eval_accuracy"]
        val_loss = metrics["eval_loss"]
        self.history["val_acc"].append(val_acc)
        self.history["val_loss"].append(val_loss)

        # Save best model
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.save_model(f"./saved_model/{model_version}{epoch+1}.0.pt")
            print(f"Saved best model at epoch {epoch+1} with val_acc={val_acc:.4f}")

    def log_training_epoch(self, train_loss, train_acc):
        self.history["train_loss"].append(train_loss)
        self.history["train_acc"].append(train_acc)

In [ ]:
EPOCHS=3

In [ ]:
optimizer_qdora = torch.optim.AdamW(model.parameters(),lr=1e-5)
total_steps = len(train_loader)*EPOCHS

scheduler_qdora = get_linear_schedule_with_warmup(
    optimizer_qdora,
    num_warmup_steps= 0,
    num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from transformers import TrainingArguments

batch_size = 32
gradient_accumulation_steps = 4

# output dir 
model_version = "securityRoBERTa_BaseQDoRA_"
model_dir = f"{model_version}"

training_args = TrainingArguments(
    run_name=model_version,
    output_dir=model_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=1,
    lr_scheduler_type="constant",
    logging_dir=f"{model_dir}/logs",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,  # Adjust based on your CPU capabilities
    gradient_checkpointing=True,  # Enable gradient checkpointing to save memory
    report_to="none",  # Disable reporting to avoid unnecessary overhead
    label_names=["labels"],
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
# The parameters after appling QDoRA
print_trainable_parameters(model)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "macro_f1": macro_f1}

# model.to(device)

Trainable parameters: 1,072,143
Total parameters: 68,101,357
Percentage of trainable params: 1.57%


In [ ]:
import json

In [ ]:
from collections import defaultdict
import time

metrics_callback = MetricsCallback()

# New trainer
trainer_qdora = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=custom_tokenizer,
    callbacks=[metrics_callback]
)

start_time = time.time()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    
    # Train for one epoch (Trainer handles batches internally)
    trainer_qdora.train()

    trainer_qdora.log_training_epoch(metrics_callback.epoch_train_losses[-1], metrics_callback.epoch_train_accuracies[-1])

    # Evaluate and potentially save best model
    trainer_qdora.evaluate_and_save(
        epoch,
        model_version
    )

history_roberta_base_qdora = trainer_qdora.history

# Save full history and training time
end_time = time.time()
history_roberta_base_qdora["training_time"].append(end_time - start_time)

torch.save(history_roberta_base_qdora, "./saved_model/history_roberta_base_qdora.pt")
print("Training complete and history saved.")



Epoch 1/3


AssertionError: 

In [ ]:
# Load
history_roberta_base_qdora = torch.load("./saved_model/history_roberta_base_qdora.pt", weights_only=False)

# Still a defaultdict with tensors
print(type(history_roberta_base_qdora))  # defaultdict
print(type(history_roberta_base_qdora['train_acc'][0]))  # torch.Tensor

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

epochs = range(1, EPOCHS+1)
train_losses = history_roberta_base_qdora['train_loss']
print(train_losses)
val_losses = history_roberta_base_qdora['val_loss']

plt.figure(figsize=(10, 6))

sns.lineplot(x=epochs, y=train_losses, label='Training Loss',marker="o")
sns.lineplot(x=epochs, y=val_losses, label='Validation Loss',marker="o")

plt.title('RoBERTa Base QDoRA Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('./roberta-base-qdora-training-validation-losses.png',dpi=780)
plt.show()

In [ ]:
epochs = range(1, EPOCHS+1)
train_accuracies = history_roberta_base_qdora['train_acc']
val_accuracies = history_roberta_base_qdora['val_acc']

plt.figure(figsize=(10, 6))

sns.lineplot(x=epochs, y=train_accuracies, label='Training accuracy',marker='o')
sns.lineplot(x=epochs, y=val_accuracies, label='Validation accuracy',marker='o')

plt.title('RoBERTa Base QDoRA Training and Validation Accuracies')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('./roberta-base-qdora-training-validation-accuracies.png',dpi=780)
plt.show()

In [ ]:
from transformers.trainer_utils import PredictionOutput
from sklearn.metrics import confusion_matrix,classification_report

def get_predictions(trainer, test_dataset):
    # Runs prediction
    output: PredictionOutput = trainer.predict(test_dataset)

    # Get raw logits and labels
    logits = torch.tensor(output.predictions)
    probs = F.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1)
    labels = torch.tensor(output.label_ids)

    return preds, probs, labels

In [ ]:
preds, probs, labels = get_predictions(trainer_qdora, test_dataset)

In [ ]:
torch.save(preds,"./saved_model/roberta_base_qdora_predictions.pt")
torch.save(probs,"./saved_model/roberta_base_qdora_predictions_probs.pt")
torch.save(labels,"./saved_model/roberta_base_qdora_real_values.pt") 

In [ ]:
target_names = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP',
       'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning',
       'Ransomware', 'SQL_injection', 'Uploading', 'Vulnerability_scanner',
       'XSS']

In [ ]:
s = set()

In [ ]:
for elt in preds:
  s.add(elt.item())

In [ ]:
actual_considered_classes = [TARGET_LIST[i] for i in s]
actual_considered_classes

In [ ]:
data['Attack_type'].value_counts()

In [ ]:
def show_confusion_matrix(confusion_matrix):

    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='d')
    plt.xticks(rotation=90)
    plt.title("RoBERTa Base QDoRA Confusion Matrix")
    plt.ylabel('Real threats')
    plt.xlabel('Predicted threats')
    plt.savefig('./roberta-base-qdora-confusion-matrix.png',dpi=780)
    plt.show()

In [ ]:
print(classification_report(labels,preds,target_names=TARGET_LIST))

In [ ]:
cm = confusion_matrix(labels,preds)
df_cm = pd.DataFrame(cm,index=TARGET_LIST,columns=TARGET_LIST)
show_confusion_matrix(df_cm)

In [ ]:
y_true = labels.cpu().numpy()
y_score = preds.cpu().numpy()